In [1]:
import numpy as np # linear algebra
import torch
import torch.nn as nn
import pandas as pd

from os.path import dirname, join as pjoin
import scipy.io as sio
mat_contents = sio.loadmat('../input/assignment889/WLDataAll.mat')

In [2]:
X = mat_contents['data']
# y = mat_contents['label']
mat_contents['label'] = mat_contents['label'].reshape(360)

y_convert = pd.get_dummies(mat_contents['label'])
# X = X.reshape(62*512,360)
X = X.reshape(360, 62*512)

y_convert = np.array(y_convert)
# y = y_convert.reshape(2,360)
y = y_convert

print(X.shape)
print(y.shape)

(360, 31744)
(360, 2)


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print("x_train shape",X_train.shape)
print("x_test shape",X_test.shape)
print("y_train shape",y_train.shape)
print("y_test shape",y_test.shape)

x_train shape (288, 31744)
x_test shape (72, 31744)
y_train shape (288, 2)
y_test shape (72, 2)


In [4]:
X_train_new = X_train.T
X_test_new = X_test.T
y_train_new = y_train.T
y_test_new = y_test.T
print("x_train shape",X_train_new.shape)
print("x_test shape",X_test_new.shape)
print("y_train shape",y_train_new.shape)
print("y_test shape",y_test_new.shape)

x_train shape (31744, 288)
x_test shape (31744, 72)
y_train shape (2, 288)
y_test shape (2, 72)


In [5]:
def sigmoid(z):
        return 1/(1+np.exp(-z)) 

def relu(Z):
    return np.maximum(0, Z)

def sigmoid_der(x):
    return sigmoid(x) *(1-sigmoid (x))

In [6]:
class NeuralNetwork:
    def __init__(self,input_size, output):
        self.INPUT_LAYER_SIZE = input_size
        self.OUTPUT_LAYER_SIZE = output
        
        self.Wh = np.full((self.INPUT_LAYER_SIZE, self.OUTPUT_LAYER_SIZE), 0)
        self.Bh = np.full((self.OUTPUT_LAYER_SIZE, 1), 0)

    
    
    def feed_forward(self, X):
        Zh = np.dot(self.Wh.T,X) + self.Bh
        H = sigmoid(Zh)
        return H
    
    def backward(self, X, y, yhat):
        o_error = yhat - y 
        dcost_wo = (1/self.INPUT_LAYER_SIZE) * X.dot(o_error.T)
        
        bias = (1/self.INPUT_LAYER_SIZE) * o_error.sum()
        
        # Update Weights ================
        self.Wh = self.Wh - 0.01 * dcost_wo
        self.Bh = self.Bh - 0.01 * bias
        
        return o_error
    
    
    def compute_accuracy(self, x_val, y_val):
        predictions = []
        total = 0
        output = self.feed_forward(x_val)
        pred = (output > 0.5).astype(int)
        acc = np.sum(pred == y_val)/(72*2)
        return acc

In [7]:
a = NeuralNetwork(31744,2)
for iteration in range(3000):
    o = a.feed_forward(np.array(X_train_new))
    error_out = a.backward(np.array(X_train_new), np.array(y_train_new), o)
    print(error_out.sum())
    accuracy = a.compute_accuracy(X_test_new, y_test_new)
    print('Accuracy: ',accuracy * 100,'%')
    if(error_out.sum() <= 0.0001 ):
        break
    print(iteration)
    

0.0
Accuracy:  16.666666666666664 %


In [8]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, AveragePooling2D, Conv2D
from sklearn.model_selection import cross_val_score, cross_validate, KFold, StratifiedKFold, cross_val_predict
from keras import layers,models
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix

In [9]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(X_train, y_train)
y_new=np.argmax(y,axis=1)
kfold = KFold(n_splits=5, shuffle=True)
accuracies = cross_val_score(estimator = neigh,
                             X = X,
                             y = y_new,
                             cv=kfold, scoring = 'accuracy')
print(accuracies.mean())

0.5


In [10]:
def make_classifier():
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu', input_dim=31744))
    model.add(layers.Dropout(rate = 0.1))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(rate = 0.1))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(rate = 0.1))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dropout(rate = 0.2))
    model.add(layers.Dense(8, activation='relu'))
    model.add(layers.Dropout(rate = 0.2))
    model.add(Flatten())
    model.add(layers.Dense(2, activation='softmax'))


    model.compile(optimizer='adam', 
                  loss='mse',
                  metrics=['accuracy'])
    return model


# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)
classifier = KerasClassifier(build_fn=make_classifier, verbose=1)

# Fit data to model
classifier.fit(X_train, y_train,batch_size=64, epochs = 50)


y_new=np.argmax(y,axis=1)

accuracies = cross_val_score(estimator = classifier,
                             X = X,
                             y = y_new,
                             cv=kfold, scoring = 'accuracy')
print(accuracies.mean())


Epoch 1/50
5/5 [==============================] - 2s 8ms/step - loss: 0.4847 - accuracy: 0.4956
Epoch 2/50
5/5 [==============================] - 0s 7ms/step - loss: 0.4374 - accuracy: 0.5269
Epoch 3/50
5/5 [==============================] - 0s 7ms/step - loss: 0.4572 - accuracy: 0.5211
Epoch 4/50
5/5 [==============================] - 0s 7ms/step - loss: 0.4472 - accuracy: 0.5365
Epoch 5/50
5/5 [==============================] - 0s 7ms/step - loss: 0.4370 - accuracy: 0.5443
Epoch 6/50
5/5 [==============================] - 0s 8ms/step - loss: 0.4620 - accuracy: 0.5343
Epoch 7/50
5/5 [==============================] - 0s 7ms/step - loss: 0.4116 - accuracy: 0.5743
Epoch 8/50
5/5 [==============================] - 0s 7ms/step - loss: 0.4685 - accuracy: 0.5232
Epoch 9/50
5/5 [==============================] - 0s 7ms/step - loss: 0.4526 - accuracy: 0.5396
Epoch 10/50
5/5 [==============================] - 0s 7ms/step - loss: 0.3924 - accuracy: 0.6022
Epoch 11/50
5/5 [======================

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


3/3 [==============================] - 0s 3ms/step
0.5166666666666666
